
# 📘 UC22 – Aula 3 — **Pandas e DataFrame** (Versão Alunos)
**Curso:** 3º Ano – Ensino Médio Técnico em Informática  
**Ferramenta:** Google Colab  
**Duração:** ~90 min  
**Tema:** Leitura de CSV e exploração inicial com **Pandas**

> “Você não precisa decorar códigos; precisa entender o que os **dados** querem te contar.” – IAra 💡



## 🎯 Objetivos de aprendizagem
- Entender com precisão **Pandas**, **DataFrame**, **Series**, **Index**, **dtypes** e **NaN**.  
- Ler CSV de forma **robusta** (encoding, separador, erros comuns).  
- Explorar um DataFrame com `head`, `tail`, `sample`, `info`, `shape`, `dtypes`, `columns`.  
- Calcular **sumários**: `describe`, `nunique`, `unique`, `value_counts`, nulos.  
- Iniciar **padronização** de colunas e textos.



## 🧠 Conceitos essenciais
**Pandas** — biblioteca Python para **análise/manipulação** de dados.  
**DataFrame** — tabela (linhas × colunas), como uma planilha **dentro do Python**.  
**Series** — uma **coluna** (ou linha) do DataFrame.  
**Index** — rótulos das linhas (por padrão: 0, 1, 2...).  
**dtypes** — tipos de dados das colunas (`int64`, `float64`, `object`, `bool`, `datetime64[ns]`).  
**NaN** — valores **ausentes** (faltantes) que precisam ser detectados e tratados mais adiante.



## 📤 Enviando os arquivos para o Colab
Você vai precisar de **dois CSVs**:
- `pokemons_pokedex.csv` *(nome alternativo comum: `pokemons.csv`)*  
- `escolas_estaduais_censo_escolar_2023.csv`

Faça o upload abaixo (Ctrl/Cmd + clique para selecionar múltiplos arquivos).


In [ ]:

# @title Upload dos arquivos CSV (Pokémon + Escolas)
# Execute esta célula, clique no botão e selecione os arquivos .csv do seu computador.
from google.colab import files  # importa utilitário do Colab para upload de arquivos

uploaded = files.upload()       # abre a janela para selecionar arquivos
# Dica: selecione "pokemons_pokedex.csv" (ou "pokemons.csv") e "escolas_estaduais_censo_escolar_2023.csv"



## 🐼 Importando o Pandas e conferindo a versão


In [ ]:

# @title Import e opções de exibição
import pandas as pd                           # 'pd' é o alias padrão para Pandas
print("Versão do Pandas:", pd.__version__)    # útil para compatibilidade e pesquisa
pd.options.display.max_columns = 50           # (opcional) mostra mais colunas ao imprimir DataFrames



## 📚 Lendo os CSVs de forma **robusta**
- **Acentos estranhos (Ã, Õ)** → ajuste o `encoding` (UTF-8 ⇄ Latin-1).  
- **Tudo em uma coluna só** → ajuste o `sep` (`,` ⇄ `;`).  
Abaixo, o código tenta os casos mais comuns para **Pokémon** e **Escolas**.


In [ ]:

# @title Leitura robusta dos CSVs
from pathlib import Path
import pandas as pd

def read_csv_resiliente(caminhos, encodings=("utf-8", "latin-1"), seps=(",", ";")):
    """
    Tenta ler um CSV testando múltiplos caminhos, encodings e separadores.
    Retorna um DataFrame lido com sucesso ou levanta FileNotFoundError/UnicodeError.
    """
    if isinstance(caminhos, (str, Path)):
        caminhos = [caminhos]

    last_error = None
    for caminho in caminhos:
        for enc in encodings:
            for sep in seps:
                try:
                    df = pd.read_csv(caminho, encoding=enc, sep=sep)
                    if df.shape[1] == 1 and sep == ",":
                        try:
                            df = pd.read_csv(caminho, encoding=enc, sep=";")
                        except Exception as e:
                            last_error = e
                    if df.shape[1] > 1:
                        return df
                except Exception as e:
                    last_error = e
    if last_error:
        raise last_error
    raise FileNotFoundError(f"Não foi possível ler o arquivo: {caminhos}")

poke_caminhos = ["pokemons_pokedex.csv", "pokemons.csv"]
df_poke = read_csv_resiliente(poke_caminhos)

escolas_caminhos = ["escolas_estaduais_censo_escolar_2023.csv"]
df_escolas = read_csv_resiliente(escolas_caminhos)

print("✅ Pokémon lido:", df_poke.shape, "→ colunas:", len(df_poke.columns))
print("✅ Escolas lido:", df_escolas.shape, "→ colunas:", len(df_escolas.columns))



## 👀 Primeira inspeção: `head`, `tail`, `sample`
Três jeitos rápidos de **ver os dados** sem rolar um arquivo gigante.


In [ ]:

# @title Primeiras visualizações (Pokémon e Escolas)
# Pokémon
display(df_poke.head())    # 5 primeiras linhas
display(df_poke.tail())    # 5 últimas linhas
display(df_poke.sample(5)) # 5 linhas aleatórias

# Escolas
display(df_escolas.head())
display(df_escolas.tail())
display(df_escolas.sample(5))



## 🧱 Estrutura do DataFrame: `shape`, `columns`, `index`, `dtypes`, `info`
- `shape`: (linhas, colunas)  
- `columns`: nomes exatos das colunas (evita erro de digitação)  
- `index`: rótulos das linhas  
- `dtypes` e `info`: tipos, não-nulos, diagnóstico rápido


In [ ]:

# @title Estrutura geral dos DataFrames
print("POKÉMON → shape:", df_poke.shape)
print("ESCOLAS  → shape:", df_escolas.shape)
print()

print("POKÉMON → columns:", list(df_poke.columns))
print("ESCOLAS  → columns:", list(df_escolas.columns))
print()

print("POKÉMON → index:", df_poke.index)
print("ESCOLAS  → index:", df_escolas.index)
print()

print("POKÉMON → dtypes:\n", df_poke.dtypes)
print()
print("ESCOLAS  → dtypes:\n", df_escolas.dtypes)
print()

print("ℹ️ POKÉMON → info()")
display(df_poke.info(verbose=True))
print("ℹ️ ESCOLAS → info()")
display(df_escolas.info(verbose=True))



## 🕳️ Valores ausentes (NaN) — contagem e porcentagem
Antes de analisar, precisamos saber **o que está faltando**.


In [ ]:

# @title Nulos por coluna (contagem e %)
import pandas as pd

nulos_poke = df_poke.isna().sum()
perc_nulos_poke = (nulos_poke / len(df_poke) * 100).round(2)
tabela_poke = pd.DataFrame({"nulos": nulos_poke, "perc_%": perc_nulos_poke}).sort_values("nulos", ascending=False)
print("NULOS — Pokémon")
display(tabela_poke)

nulos_escolas = df_escolas.isna().sum()
perc_nulos_escolas = (nulos_escolas / len(df_escolas) * 100).round(2)
tabela_escolas = pd.DataFrame({"nulos": nulos_escolas, "perc_%": perc_nulos_escolas}).sort_values("nulos", ascending=False)
print("NULOS — Escolas")
display(tabela_escolas)



## 📊 Sumários rápidos: `describe()`
Números → média, desvio, mínimos/máximos, quartis.  
Texto → contagem, valores únicos, valor mais frequente.


In [ ]:

# @title Estatísticas descritivas
print("Pokémon — numéricas")
display(df_poke.describe())

print("Pokémon — texto/categóricas")
display(df_poke.describe(include='object'))

print("Escolas — numéricas")
display(df_escolas.describe())

print("Escolas — texto/categóricas")
display(df_escolas.describe(include='object'))



## 🧮 Categorias mais frequentes: `value_counts()`
Se não souber qual coluna usar, detecte automaticamente **uma coluna de texto** e veja o **Top 10**.


In [ ]:

# @title Top 10 categorias mais frequentes (auto)
def top10_categorias(df):
    cat_cols = df.select_dtypes(include='object').columns
    if len(cat_cols) == 0:
        print("Não há colunas categóricas de texto neste DataFrame.")
        return
    col = cat_cols[0]
    print(f"Coluna categórica escolhida: {col}")
    display(df[col].value_counts().head(10))

print("Pokémon — Top 10 categorias")
top10_categorias(df_poke)

print("Escolas — Top 10 categorias")
top10_categorias(df_escolas)

# Dica: se você já sabe a coluna, use diretamente, por exemplo:
# df_escolas['NO_MUNICIPIO'].value_counts().head(10)   # (ajuste o nome conforme seu arquivo)



## 🧹 Padronização inicial (nomes e textos)
Padronizar cedo evita erros em filtros e junções.


In [ ]:

# @title Padronização de nomes de colunas e textos
df_poke.columns = (df_poke.columns
                   .str.strip()
                   .str.lower()
                   .str.replace(' ', '_', regex=False))

df_escolas.columns = (df_escolas.columns
                      .str.strip()
                      .str.lower()
                      .str.replace(' ', '_', regex=False))

if 'no_municipio' in df_escolas.columns:
    df_escolas['no_municipio'] = df_escolas['no_municipio'].astype(str).str.strip().str.title()

if 'tipo' in df_poke.columns:
    df_poke['tipo'] = df_poke['tipo'].astype(str).str.strip().str.lower()

print("✅ Padronização aplicada.")
print("POKÉMON →", list(df_poke.columns)[:12], "...")
print("ESCOLAS →", list(df_escolas.columns)[:12], "...")



## 🔢 Convertendo números que vieram como texto (seguro)
Alguns CSVs trazem números como texto (vírgula decimal, separador de milhar).  
Use a função a seguir para **tentar converter** com segurança.


In [ ]:

# @title Conversão numérica segura (vírgula decimal / milhar)
import numpy as np
import pandas as pd

def to_numeric_safe(serie):
    s = serie.astype(str).str.replace('.', '', regex=False)  # remove separador de milhar
    s = s.str.replace(',', '.', regex=False)                 # vírgula decimal → ponto
    return pd.to_numeric(s, errors='coerce')

# Exemplo de uso:
possible_numeric_cols = []  # ex.: ['qtd_alunos', 'taxa_aprovacao']
for col in possible_numeric_cols:
    if col in df_escolas.columns:
        df_escolas[col] = to_numeric_safe(df_escolas[col])
        print(f"Coluna convertida para numérico: {col}")



## 🧪 Mini-Lab — Para entregar no Classroom
**Arquivo:** `UC22_Aula03_Pratica_SeuNome.ipynb`  

1. **Mostre** as 5 primeiras e 5 últimas linhas de **cada** dataset.  
2. **Exiba**: `shape`, `columns`, `dtypes`, `info()` de cada dataset.  
3. **Calcule** nulos por coluna **e** porcentagem de nulos.  
4. **Liste** as 10 categorias mais frequentes de **uma** coluna de texto (explique sua escolha).  
5. **Padronize** nomes das colunas e uma coluna de texto relevante (ex.: município).  

> Dica: copie e adapte as células deste notebook.



## 🧯 Erros comuns e soluções rápidas
- **UnicodeDecodeError** → troque `encoding` entre `"utf-8"` e `"latin-1"`.  
- **Tudo em 1 coluna** → troque `sep` entre `","` e `";"`.  
- **Números como texto** → use a função `to_numeric_safe`.  
- **Coluna não encontrada** → confirme com `df.columns` e copie/cole o nome exato.



## 📚 Glossário
**Pandas**: biblioteca para análise de dados.  
**DataFrame**: tabela (linhas × colunas).  
**Series**: uma coluna do DataFrame.  
**Index**: rótulos das linhas.  
**dtype**: tipo de dado da coluna.  
**NaN**: valor ausente (faltando).  
**encoding**: regra de acentuação (UTF-8, Latin-1).  
**sep**: separador das colunas em CSV (`,` ou `;`).
